In [ ]:
# Standard libraries
from tabulate import tabulate
from typing import Tuple, Union
import json
import numpy as np
import os
import pandas as pd

# Plotting libraries
from bokeh.models import DataRange1d
from bokeh.plotting import figure, show
from bokeh.resources import INLINE
from bokeh.io import output_notebook
output_notebook(INLINE)

In [ ]:
def load_file(filename: str) -> Union[dict, list]:
    data_file = open(path_metrics + filename)
    data_json = json.load(data_file)
    data_file.close()
  
    return data_json


def train_val_values(filename: str):
    y_values = load_file(filename)   
    x_values = []
    x_length = len(y_values)
    
    for x in range(1, x_length + 1):
        x_values.append(x)
        
    return x_values, y_values


# Creating Settings table
def settings_table(settings: dict) -> Tuple[list, str]:
    setting_keys = list(settings.keys())
    setting_values = list(settings.values())
    setting_descr = ''
    setting_keys.insert(0, '\033[1;3m' + 'Setting' + '\033[0m')
    setting_values.insert(0, '\033[1;3m' + 'Value' + '\033[0m')

    if (setting_keys[-1].lower() == 'description') :
        setting_descr = setting_values[-1]
        setting_keys = setting_keys[:-1]
        setting_values = setting_values[:-1]

    zips_tuples = list(zip(setting_keys, setting_values))  # Output: list with tuples
    zips_lists = list(map(list, zips_tuples))  # Output: list with lists

    return zips_lists, setting_descr

In [ ]:
# Set the path to the folder which contains the metrics files
path_metrics = "../Unet_Segmentation/modelBackups/RiverBlindnessEvaluation_13_02_23_best/logs/"

# Settings

In [ ]:
settings_file = open(path_metrics + 'settings.json')
settings = json.load(settings_file)
settings_file.close()

tabledata, description = settings_table(settings)
print(tabulate(tabledata, headers='firstrow', tablefmt='fancy_grid', colalign=("left","left")))
print('\033[1;3m' + 'Description: ' + '\033[0m' + description)

# Train & Validation losses

In [ ]:
x_val, y_val = train_val_values('valLosses.json')
x_train, y_train = train_val_values('trainLosses.json')

TOOLTIPS = [
("Epoch", "@x"),
("Loss", "@y"),
]

p = figure(
    x_axis_label = "Epoch",
    y_axis_label = "Loss",
    y_range=DataRange1d(range_padding = 0.1, follow = "end", only_visible = True),
    width=800,
    height=300, 
    tooltips=TOOLTIPS,
    title = "Training and validation losses",
    )

# Training losses
p.line(x_train, y_train, legend_label = 'Training loss', color = 'blue', alpha=0.4, line_width=2)
# Validation losses
p.line(x_val, y_val, legend_label = 'Validation loss', color = 'orange', alpha=0.7, line_width=2)

p.yaxis.minor_tick_line_color = None
p.legend.click_policy="hide"

show(p)

print("Mean training loss: " + "\t" + str(np.mean(y_train)))
print("Mean validation loss: " + "\t" + str(np.mean(y_val))) 

y_train_meaned = []
y_val_meaned = []

for y in range(len(y_train)):
    sliced_y_train = y_train[:y+1:]
    y_train_meaned.append(np.mean(sliced_y_train))

for y in range(len(y_val)):
    sliced_y_val = y_val[:y+1:]
    y_val_meaned.append(np.mean(sliced_y_val))